<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width = 400, align = "center"></a>

# <center>Decision Trees</center>

En este ejercicio, aprenderás un algoritmo muy popular de machine learning llamado Árboles de Decisión. Utilizarás un algoritmo de clasificación para construir un modelo basado en datos históricos de pacientes y sus respectivos medicamentos. Luego, utilizarás el árbol de decisión recién entrenado para predecir la clase de paciente desconocido o para encontrar la droga adecuada para el mismo.

Import the Following Libraries:
<ul>
    <li> <b>numpy (as np)</b> </li>
    <li> <b>pandas</b> </li>
    <li> <b>DecisionTreeClassifier</b> from <b>sklearn.tree</b> </li>
</ul>

In [1]:
import numpy as np 
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

### Acerca del set de datos
Imagina que eres un investigador médico recolectando datos para un estudio. Has colectado datos de un grupo de pacientes, todos sufrieron la misma enfermedad. Durante su tratamiento, cada paciente respondio a una de 5 medicaciones, Droga A, Droga B, Droga c, Droga x e y. 

Parte de tu trabajo es construir un modelo para encontrar la droga apropiada para un próximo paciente con la misma enfermedad. El conjunto de características son Edad, Sexo, Presión Sanguínea y Colesterol. El objetivo es la droga ante la cual cada paciente respondió. 

Este es un ejemplo de un clasificador binario donde puedes utilizar un set de entrenamiento del set de datos 
para construir un árbol de decisión para predecir la clase de pacientes desconocidos o para prescribirle a un nuevo paciente.


### Descargando los Datos
Para descagar los datos, utilizaremos !wget desde IBM Object Storage.

In [ ]:
!wget -O drug200.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/drug200.csv

__¿Sabías?__ Cuando se trata de Machine Learning, seguro trabajarás con grandes datasets (juego de datos). Entonces, ¿dónde podrás guardar esos datos? IBM ofrece una oportunidad única para las empresas, con 10 Tb de IBM Cloud Object Storage: [Regístrate ahora gratuitamente](http://cocl.us/ML0101EN-IBM-Offer-CC)

ahora, lee los datos utilizando el marco de datos de panda:

In [2]:
my_data = pd.read_csv(r"..\drug200.csv", delimiter=",")
my_data[0:5]

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,drugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,drugY


In [7]:
my_data.dtypes

Age              int64
Sex             object
BP              object
Cholesterol     object
Na_to_K        float64
Drug            object
dtype: object

## Práctica 
¿Cuál es el tamaño de los datos? 

In [3]:
# escribe tu código aquí
my_data.shape

(200, 6)

## Pre-procesamiento

Utilizando <b>my_data</b> como los datos de panda el archivo Drug.csv, declara las siguientes variables: <br>
<ul>
    <li> <b> X </b> as the <b> Feature Matrix </b> (datos de my_data) </li>
    <li> <b> y </b> como el <b> vector de respuesta (target) </b> </li>
  
</ul>

Elimina la columna que contiene el target ya que no posee valores numéricos.

In [10]:
X = my_data[['Age', 'Sex', 'BP', 'Cholesterol', 'Na_to_K']].values
X[0:5]

array([[23, 'F', 'HIGH', 'HIGH', 25.355],
       [47, 'M', 'LOW', 'HIGH', 13.093],
       [47, 'M', 'LOW', 'HIGH', 10.113999999999999],
       [28, 'F', 'NORMAL', 'HIGH', 7.797999999999999],
       [61, 'F', 'LOW', 'HIGH', 18.043]], dtype=object)

Como te puedes imaginar, algunas características son de categoría, tales como __Sex__ o __BP__. Desafortunadamente, los árboles de Decisión Sklearn no manejan variables categóricas. Pero las podemos convertir en valores numéricos. __pandas.get_dummies()__
Convertir variable categórica en indicadores de variables.

In [15]:
categorical = {1:'Sex', 2: 'BP', 3:'Cholesterol'}
for k1, item in categorical.items():
    print(item)
    print(my_data[item].value_counts())
    print()

Sex
M    104
F     96
Name: Sex, dtype: int64

BP
HIGH      77
LOW       64
NORMAL    59
Name: BP, dtype: int64

Cholesterol
HIGH      103
NORMAL     97
Name: Cholesterol, dtype: int64



In [16]:
from sklearn import preprocessing
le_sex = preprocessing.LabelEncoder()
le_sex.fit(['F','M'])
X[:,1] = le_sex.transform(X[:,1]) 


le_BP = preprocessing.LabelEncoder()
le_BP.fit([ 'LOW', 'NORMAL', 'HIGH'])
X[:,2] = le_BP.transform(X[:,2])


le_Chol = preprocessing.LabelEncoder()
le_Chol.fit([ 'NORMAL', 'HIGH'])
X[:,3] = le_Chol.transform(X[:,3]) 

X[0:5]


array([[23, 0, 0, 0, 25.355],
       [47, 1, 1, 0, 13.093],
       [47, 1, 1, 0, 10.113999999999999],
       [28, 0, 2, 0, 7.797999999999999],
       [61, 0, 1, 0, 18.043]], dtype=object)

Ahora, podemos completar la variable objetivo (target).

In [19]:
y = my_data["Drug"]
my_data['Drug'].value_counts()

drugY    91
drugX    54
drugA    23
drugB    16
drugC    16
Name: Drug, dtype: int64

---
## Configurando el Arbol de Decisión
Estaremos utilizando <b>entrenar/probar separar</b> en nuestro <b>árbol de decisión</b>. Importemos <b>train_test_split</b> de <b>sklearn.cross_validation</b>.

In [20]:
from sklearn.model_selection import train_test_split

Ahora <b> train_test_split </b> devolverá 4 parámetros diferentes. Los nombraremos:<br>
X_trainset, X_testset, y_trainset, y_testset <br> <br>
El <b> train_test_split </b> necesitará los parámetros: <br>
X, y, test_size=0.3, and random_state=3. <br> <br>
La <b>X</b> e <b>y</b> son los arreglos necesarios ántes de la operación dividir/separar, <b>test_size</b> representa el grado del dataset de pruebas, y el <b>random_state</b> asegura que obtendremos las mismas divisiones.

In [26]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3, random_state=3)

## Práctica
Imprimir la forma X_trainset e y_trainset. Asegurarse que las dimensiones coincidan

In [29]:
# your code
print('X_trainset', X_trainset.shape)
print('y_trainset', y_trainset.shape)


X_trainset (140, 5)
y_trainset (140,)


Imprimir la forma de X_testset e y_testset. Asegurarse que las dimensiones coincidan

In [31]:
# tu código
print('X_testset', X_testset.shape)
print('y_testset', y_testset.shape)


X_testset (60, 5)
y_testset (60,)


## Modelando
Primero crearemos una instancia del <b>DecisionTreeClassifier</b> llamada <b>drugTree</b>.<br>
Dentro del clasificador, especificaremos <i> criterion="entropy" </i> para que podamos ver la nueva información de cada nodo.

In [32]:
drugTree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
drugTree # muestra los parámetros por omisión

DecisionTreeClassifier(criterion='entropy', max_depth=4)

Luego, adaptaremos los datos con la matriz de entrenamiento <b> X_trainset </b> y el vector de respuesta <b> y_trainset </b>

In [33]:
drugTree.fit(X_trainset,y_trainset)

DecisionTreeClassifier(criterion='entropy', max_depth=4)

## Predicción
Ahora hagamos algunas <b>predicciones</b> en el dataset de pruebas y guardémoslas en una variable llamada <b>predTree</b>.

In [34]:
predTree = drugTree.predict(X_testset)

Puedes imprimir <b>predTree</b> y <b>y_testset</b> si quieres comparar visualmente la predicción con los valores actuales.

In [48]:
print('predTree, y_testset, test')
for predT, item in zip(predTree[0:5], y_testset[0:5]):
    print(predT, item, predT == item )
print()
print('i, y_testset')
print (y_testset [0:5])


predTree, y_testset, test
drugY drugY True
drugX drugX True
drugX drugX True
drugX drugX True
drugX drugX True

i, y_testset
40     drugY
51     drugX
139    drugX
197    drugX
170    drugX
Name: Drug, dtype: object


## Evaluación
Luego, importemos __metrics__ de sklearn y revisemos la precisión de nuestro modelo.

In [49]:
from sklearn import metrics
import matplotlib.pyplot as plt
print("Precisión de los Arboles de Decisión: ", metrics.accuracy_score(y_testset, predTree))

Precisión de los Arboles de Decisión:  0.9833333333333333


__Accuracy classification score__ calcula la precisión del subconjunto: las etiquetas predichas para una muestra deben coincidir con las correspondientes etiquetas en y_true.  

En la clasificación multietiqueta, la función devuelve un subconjunto de precisión. Si el conjunto de etiquetas predichas para una muestra coincide totalmente con el conjunto de etiquetas, entonces la precisión del subconjunto es 1.0; de no ser así, es 0.0.


## Práctica 
¿Puedes calcular la precisión sin sklearn ?

In [54]:
# tu código aquí
# Jacard index
nTruePred = 0
for predT, item in zip(predTree, y_testset):
    if predT == item:
        nTruePred += 1
npredTree = predTree.shape[0]
ny_testset = y_testset.shape[0]
Jaccard = nTruePred / (npredTree + ny_testset - nTruePred )
print('Jaccard index ', Jaccard)

Jaccard index  0.9672131147540983


## Visualización
Observemos el árbol

In [ ]:
from sklearn.externals.six import StringIO
import pydotplus
import matplotlib.image as mpimg
from sklearn import tree
%matplotlib inline 

In [ ]:
dot_data = StringIO()
filename = "drugtree.png"
featureNames = my_data.columns[0:5]
targetNames = my_data["Drug"].unique().tolist()
out=tree.export_graphviz(drugTree,feature_names=featureNames, out_file=dot_data, class_names= np.unique(y_trainset), filled=True,  special_characters=True,rotate=False)  
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize=(100, 200))
plt.imshow(img,interpolation='nearest')

## ¿Deseas aprender más?

IBM SPSS Modeler es una plataforma para analytics que contiene varios algoritmos de machine learning. Fue diseñada para acercar inteligencia predictiva a las decisiones hechas por individuos, grupos, sistemas, toda la empresa. Un free trial está disponible a través de este curso en: [SPSS Modeler](http://cocl.us/ML0101EN-SPSSModeler).

Asi mismo, puedes utilizar Watson Studio para ejecutar estos notebooks más rápido y con datasets más grandes. Watson Studio es una solución en la nube lider de IBM's para científicos de datos, construída por científicos de datos. Con Jupyter notebooks, RStudio, Apache Spark y librerías conocidas pre instaladas en la nube, Watson Studio posibilita a los científicos de datos colaborar en sus proyectos sin tener que instalar nada. Sumate a la comunidad de usuarios Watson Studio hoy mismo por medio de una cuenta gratuita en [Watson Studio](https://cocl.us/ML0101EN_DSX)

### ¡Gracias por completar esta lección!

Laboratorio creado por: <a href = "https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>

<hr>
Copyright &copy; 2018 [Cognitive Class](https://cocl.us/DX0108EN_CC). Este lab y su código fuente fueron registrados bajo los términos de [MIT License](https://bigdatauniversity.com/mit-license/).​